<a href="https://colab.research.google.com/github/NamDo8467/101293009_COMP3123_Assignment1/blob/main/Web_Scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Useful links
[Reddit API tutorial](https://www.youtube.com/watch?v=x9boO9x3TDA)

[Reddit API Documentation](https://www.reddit.com/dev/api)

[Reddit App](https://www.reddit.com/prefs/apps)

[Reddit OAuth2 Github repo](https://github.com/reddit-archive/reddit/wiki/OAuth2-Quick-Start-Example)

# Code

In [ ]:
from bs4 import BeautifulSoup
import requests

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
# Get access_token from Reddit API
import requests
client_auth = requests.auth.HTTPBasicAuth('lwl5lxEHUx_QqUF5gIgVKg', 'jHn1pV9hKk7vAesM_mlGZAaEj-2QEQ')
post_data = {"grant_type": "password", "username": "NamDough", "password": "Namdo2002"}
headers = {"User-Agent": "ChangeMeClient/0.1 by YourUsername"}
response = requests.post("https://www.reddit.com/api/v1/access_token", auth=client_auth, data=post_data, headers=headers)

access_token = response.json()['access_token']



In [2]:
# Get all the subreddits from a user
headers = {"Authorization": f"bearer {access_token}", "User-Agent": "ChangeMeClient/0.1 by YourUsername"}
response1 = requests.get("https://oauth.reddit.com/subreddits/mine/subscriber", headers=headers)


In [ ]:
import pandas as pd


In [3]:
import pandas as pd

spreadsheet_id = "1Aghg_s9fVC7lDJCdjzs3-JTff_YXJkzM3rUC4B9h9og"

link_df = pd.read_csv(f"https://docs.google.com/spreadsheets/d/{spreadsheet_id}/export?format=csv")

print(link_df)

subreddit_column = link_df.loc[:, "Subreddit"]

link_column = link_df.loc[:, "Link"]

links = {
    'user-info': "https://oauth.reddit.com/api/v1/me",
    'user-karma':"https://oauth.reddit.com/api/v1/me/karma",
    'user-friends':"https://oauth.reddit.com/prefs/friends",
    'subreddit':{
        'Tim Hortons':[],
        'York University':[],
        'Toronto':[],
        'Web Dev':[],
        'Python':[]
    }
}
# for index, link in enumerate(link_column):
#   if subreddit_column[index] not in links['subreddit']:
#     links["subreddit"][f'{subreddit_column[index]}'] = []
#   else:
#     links["subreddit"][f'{subreddit_column[index]}'].append(link)


         Subreddit                                               Link
0          Toronto  https://www.reddit.com/r/toronto/comments/1eby...
1          Toronto  https://www.reddit.com/r/toronto/comments/1eak...
2          Toronto  https://www.reddit.com/r/toronto/comments/1ebr...
3      Tim Hortons  https://www.reddit.com/r/TimHortons/comments/1...
4  York University  https://www.reddit.com/r/yorku/comments/1edwen...


In [4]:
# Get all comments from a specific post
headers = {"Authorization": f"bearer {access_token}", "User-Agent": "ChangeMeClient/0.1 by YourUsername"}
response1 = requests.get("https://oauth.reddit.com/r/TimHortons/comments/1earicv/job_interview/", headers=headers)

res = response1.json() # res[0] is the information about the post, and res[1] is the information about the comments of the post

post_info = res[0]
comments = res[1].get("data").get("children")


comment_arr = []

subreddit_arr = []

column_name = ["Comment", "Subreddit"]
for comment in comments:
  data = comment.get('data')
  comment_and_subreddit = [data.get("body"), data.get("subreddit")]
  comment_arr.append(comment_and_subreddit)

df = pd.DataFrame(comment_arr, columns=column_name)

print(df)




                                              Comment   Subreddit
0   I’ve asked silly questions like this to people...  TimHortons
1   Not specific to Tim Hortons, but as someone wh...  TimHortons
2   I was asked if I was staring at interviewers t...  TimHortons
3   I’ve asked these types of questions while inte...  TimHortons
4   I was once asked if I could have any job in th...  TimHortons
5   If you were a tree what kind of tree would you...  TimHortons
6   The questions weren’t that deep bro. The super...  TimHortons
7   When I applied for my job I had to make a vide...  TimHortons
8   During an interview at McDonald’s when I was 1...  TimHortons
9                      Yall complain about anything 🤣  TimHortons
10  I find all these big brand companys always cha...  TimHortons


In [7]:
# Using model (Distilbert) to classfy the text sentiment
from transformers import pipeline
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

classification_machine = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)


In [17]:
# Get the comments from df
texts = list(df["Comment"])

results = classification_machine(texts)


for text, result in zip(texts, results):
  print(f"Text: {text}")
  print("Result: ", result)

Text: I’ve asked silly questions like this to people. I’ve already made up my mind if I was hiring them before the interview.
Result:  {'label': 'NEGATIVE', 'score': 0.9945656061172485}
Text: Not specific to Tim Hortons, but as someone who has done a lot of interviews before, it can often be really hard to get a feel for someone’s actual personality during an interview, which is especially important for customer service jobs.  

These type of questions are intended to throw you off, in a good way, break the tension, and also see what sort of personality and charisma you may have, to react to a question that you almost certainly wouldn’t have prepared for in advance.
Result:  {'label': 'POSITIVE', 'score': 0.9482031464576721}
Text: I was asked if I was staring at interviewers tits once lol.
Result:  {'label': 'NEGATIVE', 'score': 0.9973764419555664}
Text: I’ve asked these types of questions while interviewing, it gives me an idea if they’re able to think on their feet in a sense. Plus i

In [ ]:
# User Roberta model to classify the text sentiment
from transformers import pipeline
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification

MODEL = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)

model = AutoModelForSequenceClassification.from_pretrained(MODEL)

roberta_machine = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)



In [23]:
result = roberta_machine(["During an interview at McDonald’s when I was 15, I was asked what item I would want if I were trapped on a deserted island. I think it’s just to get an idea of your personality"])
print(result)

[{'label': 'neutral', 'score': 0.8059695959091187}]


In [ ]:


# headers = {"Authorization": f"{access_token}", "User-Agent": "ChangeMeClient/0.1 by YourUsername"}
response = requests.get("https://oauth.reddit.com/api/v1/me", headers=headers)

# response.text
response.json()
